In [1]:
# Importing packages
import numpy as np
from tqdm import tqdm
import itertools
from sympy import solve, symbols, cos, sin

# Importing Pauli matrices
from pauli_matrices.pauli_matrices import I, X, Y, Z

# Importing functions
from hamiltonian.hamiltonian import get_hamiltonian
from Bell_operator.Bell_operator import get_Bell_operator
from Bell_operator.coefficients import get_coefficients, get_system_of_equations
from classical_optimization.classical_optimization import classical_optimization
from correlation_matrix.correlation_matrix import get_correlation_matrix

In [3]:
# Defining the number of qubits
N = 2

# Defining the Hamiltonian
H = np.sqrt(2)*(np.kron(Z, Z) + np.kron(X, X))

# Calculating the eigenvalues and eigenstates
eig_vals, eig_vecs = np.linalg.eigh(H)

# extracting the ground state energy and the respective eigenstate
psi_G = eig_vecs[:,0]
psi_G_dagger = np.conjugate(np.transpose(psi_G))

In [4]:
# Defining the number of measurements
m = 2

# We do need to define some constraints on the measurements.
# Maybe we can make it so we always have each party have an angle of pi/2 between their measurements
# This makes it so we only have one effective parameter, the relative angle between the two measurements

# Defining meaurement angles
# theta = [0, np.pi/40, np.pi/100, -np.pi/100]
theta = [np.pi/4, -np.pi/2, np.pi/2, -np.pi/4]
theta = np.reshape(theta, (N, m))

# Obtaining the correlation matrix
M, variables = get_correlation_matrix(N, m)

# Obtaining the Bell operator in its matrix form and the coefficients as sympy objects
B, var_dict = get_Bell_operator(M, m, N)

# Obtaining the system of equations and solving them
var_dict = get_coefficients(B, H, var_dict, N)

# Inserting the angles into the coefficients
for key in var_dict.keys():
    for var in variables.keys():
        if var[0] == 'x':
            var_dict[key] = var_dict[key].subs( variables[var], np.cos( theta[int(var[3]), int(var[4])] ) )
        elif var[0] == 'y':
            var_dict[key] = var_dict[key].subs( variables[var], np.sin( theta[int(var[3]), int(var[4])] ) )

# Obtaining the classical bound with the recursive algorithm
beta_C = classical_optimization(var_dict, N, m)
beta_C

-6.82842712474619

In [348]:
var_dict

{'c_{00}': 0.0,
 'c_{01}': 0.0,
 'c_{02}': 0.0,
 'c_{10}': 0.0,
 'c_{11}': 7071.30351883998,
 'c_{12}': -7071.30351883998,
 'c_{20}': 0.0,
 'c_{21}': -7070.24281447240,
 'c_{22}': 7071.65709874840}

In [315]:
# Defining the number of measurements
m = 2

# Defining meaurement angles
theta = [0, np.pi/2, np.pi/6, -np.pi/8]
theta = np.reshape(theta, (N, m))

# Defining measurement angles
indices = list(itertools.product([i for i in range(m)], repeat=N))

# Obtaining variables in front of the coefficients
variables = {}
for idx in indices:

    # Obtaining the variable indices
    lower_string = ''

    for j in idx:
        lower_string += str(j)

    # Finilizing the variable
    var_name = 'theta_{' + lower_string + '}'
    variables[var_name] = symbols(var_name)

# Initializing the correlation matrix
M = [ [ [] for _ in range(m+1) ] for _ in range(N) ]

# Computing its elements
for i in range(N):
    for j in range(m+1):
        if j == 0:
            M[i][j] = I
        else:
            M[i][j] = cos(variables['theta_{'+str(i)+str(j-1)+'}'])*X + sin(variables['theta_{'+str(i)+str(j-1)+'}'])*Z

# Converting the correlation matrix to a numpy array
M = np.array(M)

# Obtaining the Bell operator in its matrix form and the coefficients as sympy objects
B, var_dict = get_Bell_operator(M, m, N)

# Obtaining the system of equations and solving them
var_dict = get_coefficients(B, H, var_dict, N)

# Inserting the angles into the coefficients
for key in var_dict.keys():
    for var in variables.keys():
        var_dict[key] = var_dict[key].subs( variables[var], theta[int(var[-3]), int(var[-2])] ) 

# Obtaining the classical bound with the recursive algorithm
beta_C = classical_optimization(var_dict, N, m)
beta_C

-3.39976953562480

In [316]:
var_dict

{'c_{00}': 0.0,
 'c_{01}': 0.0,
 'c_{02}': 0.0,
 'c_{10}': 0.0,
 'c_{11}': 0.682162754804218,
 'c_{12}': 0.891288591445235,
 'c_{20}': 0.0,
 'c_{21}': 1.64688657439413,
 'c_{22}': -1.54375712458965}

In [276]:
np.sqrt(2)*np.sin(np.pi/8)/(np.sin(np.pi/8)*np.cos(np.pi/8)+np.cos(np.pi/8)*np.sin(np.pi/6))

0.6636427241614022

In [10]:
# Obtaining the Bell operator 
for i in range(2**N):
    for j in range(2**N):
        for variable in var:
            B[i,j] = B[i,j].subs(variable, ans[variable])

# Calculting the quantum value
beta_Q = np.matmul(psi_G_dagger, np.matmul(B, psi_G))


In [7]:
print(r'beta_Q = %.3f and beta_C = %.3f' %(beta_Q, beta_C))

beta_Q = -2.000 and beta_C = 2.828
